In [1]:
# Setup the Jupyter version of Dash
import math
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html, callback_context
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
from dotenv import load_dotenv
# JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from Animal_Shelter_DB_CRUD_Python_Module import Animal_Shelter

###########################
# Data Manipulation / Model
###########################
load_dotenv()

# Update with your username and password and CRUD Python module name
username = os.environ.get("MONGO_USER")
password = os.environ.get("MONGO_PASS")
host = os.environ.get("MONGO_HOST", "127.0.0.1")
port = int(os.environ.get("MONGO_PORT", "27017"))
database = os.environ.get("MONGO_DB", "aac")
collection = os.environ.get("MONGO_COL", "animals")
auth_source = os.environ.get("MONGO_AUTHSOURCE", database)

# Connect to database via CRUD Module
db = Animal_Shelter(username, password, host, port, database, collection, auth_source=auth_source)

projection = {
    "_id": 0,
    "animal_type": 1,
    "breed": 1,
    "sex_upon_outcome": 1,
    "age_upon_outcome_in_weeks": 1,
    "name": 1,
    "location_lat": 1,
    "location_long": 1
}

records = db.read({}, projection=projection, limit=100)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(records)

FILTER_QUERIES = {
    "Water Rescue": {
        "animal_type": "Dog",
        "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
        "sex_upon_outcome": "Intact Female",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
    },
    "Mountain Rescue": {
        "animal_type": "Dog",
        "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
    },
    "Disaster Rescue": {
        "animal_type": "Dog",
        "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
    },
    "Reset": {}
}

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),    # Load in Image
    html.Center(html.B(html.H1('Animal Shelter Dashboard'))),
    html.Center(html.H4('Created by: Eric Foster')),    # Unique identifier
    html.Hr(),
    html.Div([
        # Added filtering options using an in-line radio button setup
        dcc.RadioItems(
            id = 'filter-type',
            options = ['Water Rescue', 'Mountain Rescue', 'Disaster Rescue', 'Reset'],
            value = 'Reset',
            inline = True
        )
    ]),
    html.Hr(),
    # Updated table to include pages and page scrolling for a better user experience
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_action="custom",
        page_current=0,
        page_size=10,
        page_count=0,
        filter_action="none",
        sort_action="none",
        sort_mode="multi",
        row_selectable="single",
        selected_rows=[0],
    ),
    html.Br(),
    html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
        style={'display' : 'flex'},
            children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Update data table based on selected filter option
@app.callback(
    Output("datatable-id", "data"),
    Output("datatable-id", "page_count"),
    Output("datatable-id", "page_current"),
    Input("filter-type", "value"),
    Input("datatable-id", "page_current"),
    Input("datatable-id", "page_size"),
)
def update_table(filter_type, page_current, page_size):
    query = FILTER_QUERIES.get(filter_type, {})

    # If filter changed, force back to page 0
    triggered = callback_context.triggered[0]["prop_id"].split(".")[0] if callback_context.triggered else None
    if triggered == "filter-type":
        page_current = 0

    total_rows = db.count(query)                 # <-- requires the new count() method
    page_count = max(1, math.ceil(total_rows / page_size))

    # Clamp in case the current page is now out of range
    if page_current >= page_count:
        page_current = page_count - 1

    skip = page_current * page_size

    records = db.read(
        query,
        projection=projection,
        limit=page_size,
        skip=skip
    )

    return records, page_count, page_current

# Display the breeds of animal based on quantity represented in the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('filter-type', "value")]
)
def update_graphs(filter_type):
    query = FILTER_QUERIES.get(filter_type, {})
    counts = db.breed_counts(query, limit=20)

    if not counts:
        return [html.P("No breed data available for the selected filter.")]

    dff = pd.DataFrame(counts)

    fig = px.pie(
        dff,
        names="breed",
        values="count",
        title=f"Preferred Breeds for Selected Rescue Type: {filter_type if filter_type != 'Reset' else 'All Animals'}"
    )

    return [dcc.Graph(figure=fig)]

    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output("datatable-id", "style_data_conditional"),
    Input("datatable-id", "selected_columns"),
)
def update_styles(selected_columns):
    selected_columns = selected_columns or []  # turns None into []
    return [
        {"if": {"column_id": col}, "background_color": "#D2F3FF"}
        for col in selected_columns
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
    Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, selected_rows):
    if not viewData:
        return [html.P("No data available to map.")]

    dff = pd.DataFrame.from_records(viewData)

    # If nothing is selected, default to first row
    row = 0
    if selected_rows and len(selected_rows) > 0:
        row = selected_rows[0]

    # Guard: required columns must exist
    required = {"location_lat", "location_long", "breed", "name"}
    if not required.issubset(set(dff.columns)):
        return [html.P(f"Missing map fields. Need: {sorted(required)}")]

    lat = dff.loc[row, "location_lat"]
    lon = dff.loc[row, "location_long"]

    # Guard: lat/lon must be valid numbers
    if pd.isna(lat) or pd.isna(lon):
        return [html.P("Selected record does not have valid GPS coordinates.")]

    breed = dff.loc[row, "breed"]
    name = dff.loc[row, "name"]

    return [
        dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[lat, lon],   # center on selected animal
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[lat, lon],
                    children=[
                        dl.Tooltip(str(breed)),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(str(name))
                        ])
                    ]
                )
            ]
        )
    ]

# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
app.run_server() 

Dash app running on http://127.0.0.1:8050/
